# Create and Invoke Agent via Boto3 SDK

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to use the `bedrock-agent` and the `bedrock-agent-runtime` boto3 clients to:
- create an agent
- create and action group
- associate the agent with the action group and prepare the agent
- create an agent alias
- invoke the agent

We will use Bedrock's Claude v2.1 using the Boto3 API. 

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Pre-requisites
This notebook requires permissions to: 
- create and delete Amazon IAM roles
- create, update and invoke AWS Lambda functions 
- create, update and delete Amazon S3 buckets 
- access Amazon Bedrock 

If you are running this notebook without an Admin role, make sure that your role include the following managed policies:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonS3FullAccess
- AmazonBedrockFullAccess

And the following custom policy for Amazon OpenSearch Serverless access (required for Lab 7.2)

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "aoss:*",
            "Resource": "*"
        }
    ]
}
```

#### Context
We will demonstrate how to create and invoke an agent for Bedrock using the Boto3 SDK

#### Use case
For this notebook, we use an insurance claims use case to build our Agent. The agent helps the insurance provider checking the open claims, identifying the details for a specific claim, get open documents for a claim and send reminders for a claim policyholder. The following diagram illustrates the sample process flow.

![sequence-flow-agent](images/92-agent-workflow.png)

#### Architecture
The following diagram depicts a high-level architecture of this solution.

![architecture-diagram](images/92-agent-architecture.png)

The Agent created can handle the follow tasks:
- Get Open Claims
- Get Claim Details
- Get Claim Outstanding Documents
- Send Claim reminder

## Notebook setup
Before starting, let's import the required packages and configure the support variables

In [1]:
import os 

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"
os.environ["AWS_PROFILE"] = "pcore-genai"

In [2]:

%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.34.28" \
    "awscli>=1.29.57" \
    "botocore>=1.34.28"

  Obtaining dependency information for boto3>=1.34.28 from https://files.pythonhosted.org/packages/4f/bc/ceb6631fb8e343d61d8fbeaddf1c50f4250ebe8ca1e6079cb42b2227a48a/boto3-1.34.28-py3-none-any.whl.metadata
  Using cached boto3-1.34.28-py3-none-any.whl.metadata (6.6 kB)
  Obtaining dependency information for awscli>=1.29.57 from https://files.pythonhosted.org/packages/f7/75/e3793a2f35b241d146262b4886ddbb711566ffcc7d32abebc0d81e47d18a/awscli-1.32.28-py3-none-any.whl.metadata
  Using cached awscli-1.32.28-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for botocore>=1.34.28 from https://files.pythonhosted.org/packages/7e/5f/209529304f0d6f41214ef128e9d781d97a854e4e9317207d8a79b948a2b0/botocore-1.34.28-py3-none-any.whl.metadata
  Using cached botocore-1.34.28-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Obtaining dependency information for s3transfer<0.11.0,>=0.10.0 from https://files.pythonhosted.org/packages/12/bb/7e7912e

In [3]:
import logging
import boto3
import random
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint

In [4]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

[2024-01-26 13:55:14,523] p70823 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [6]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '626103420214')

In [7]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
agent_name = "insurance-claims-agent"
agent_alias_name = "workshop-alias"
bucket_name = f'{agent_name}-{suffix}'
bucket_key = f'{agent_name}-schema.json'
schema_name = 'insurance_claims_agent_openapi_schema.json'
schema_arn = f'arn:aws:s3:::{bucket_name}/{bucket_key}'
bedrock_agent_bedrock_allow_policy_name = f"{agent_name}-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"{agent_name}-s3-allow-{suffix}"
lambda_role_name = f'{agent_name}-lambda-role-{suffix}'
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{suffix}'
lambda_code_path = "lambda_function.py"
lambda_name = f'{agent_name}-{suffix}'

### Create S3 bucket and upload API Schema

Agents require an API Schema stored on s3. Let's create an S3 bucket to store the file and upload the file to the newly created bucket

In [8]:
# Create S3 bucket for Open API schema
s3bucket = s3_client.create_bucket(
    Bucket=bucket_name
)

In [9]:
# Upload Open API schema to this s3 bucket
s3_client.upload_file(schema_name, bucket_name, bucket_key)

### Create Lambda function for Action Group
Let's now create the lambda function required by the agent action group. We first need to create the lambda IAM role and it's policy. After that, we package the lambda function into a ZIP format to create the function

In [10]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_role_name)

iam_client.attach_role_policy(
    RoleName=lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': '2cd00cd0-1d84-4730-b14d-1fecaa29e0ed',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2cd00cd0-1d84-4730-b14d-1fecaa29e0ed',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Fri, 26 Jan 2024 18:57:18 GMT'},
  'RetryAttempts': 0}}

In [11]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(lambda_code_path)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

### Create Agent
We will now create our agent. To do so, we first need to create the agent policies that allow bedrock model invocation  and s3 bucket access. 

In [12]:
# Create IAM policies for agent

bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/anthropic.claude-v2:1"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [13]:
bedrock_agent_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAgentAccessOpenAPISchema",
            "Effect": "Allow",
            "Action": ["s3:GetObject"],
            "Resource": [
                schema_arn
            ]
        }
    ]
}


bedrock_agent_s3_json = json.dumps(bedrock_agent_s3_allow_policy_statement)
agent_s3_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_s3_allow_policy_name,
    Description=f"Policy to allow invoke Lambda that was provisioned for it.",
    PolicyDocument=bedrock_agent_s3_json
)

In [14]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_s3_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'd81809d3-45a1-474f-b22c-fb964753c6b7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd81809d3-45a1-474f-b22c-fb964753c6b7',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Fri, 26 Jan 2024 18:57:45 GMT'},
  'RetryAttempts': 0}}

#### Creating Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [15]:
# Create Agent
agent_instruction = """
You are an agent that can handle various tasks related to insurance claims, including looking up claim 
details, finding what paperwork is outstanding, and sending reminders. Only send reminders if you have been 
explicitly requested to do so. If an user asks about your functionality, provide guidance in natural language 
and do not include function names on the output."""

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent for handling insurance claims.",
    idleSessionTTLInSeconds=1800,
    foundationModel="anthropic.claude-v2:1",
    instruction=agent_instruction,
)

Looking at the created agent, we can see its status and agent id

In [16]:
response

{'ResponseMetadata': {'RequestId': '55dc7af9-dae2-40a7-aa7e-b943b15cd55b',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 26 Jan 2024 18:58:33 GMT',
   'content-type': 'application/json',
   'content-length': '890',
   'connection': 'keep-alive',
   'x-amzn-requestid': '55dc7af9-dae2-40a7-aa7e-b943b15cd55b',
   'x-amz-apigw-id': 'SKUR_FaoIAMEN_A=',
   'x-amzn-trace-id': 'Root=1-65b400d9-2e1d47d74173ec4e61815a67'},
  'RetryAttempts': 0},
 'agent': {'agentId': 'JBHDO979WZ',
  'agentName': 'insurance-claims-agent',
  'agentArn': 'arn:aws:bedrock:us-east-1:626103420214:agent/JBHDO979WZ',
  'instruction': '\nYou are an agent that can handle various tasks related to insurance claims, including looking up claim \ndetails, finding what paperwork is outstanding, and sending reminders. Only send reminders if you have been \nexplicitly requested to do so. If an user asks about your functionality, provide guidance in natural language \nand do not include function names on the output.',
 

Let's now store the agent id in a local variable to use it on the next steps

In [17]:
agent_id = response['agent']['agentId']
agent_id

'JBHDO979WZ'

### Create Agent Action Group
We will now create and agent action group that uses the lambda function and API schema files created before.
The `create_agent_action_group` function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet create an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In [18]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName='ClaimManagementActionGroup',
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    description='Actions for listing claims, identifying missing paperwork, sending reminders'
)

In [19]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '56a5cad6-b504-4bdb-9e31-1424c05dcae5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 26 Jan 2024 18:59:16 GMT',
   'content-type': 'application/json',
   'content-length': '619',
   'connection': 'keep-alive',
   'x-amzn-requestid': '56a5cad6-b504-4bdb-9e31-1424c05dcae5',
   'x-amz-apigw-id': 'SKUYsEmwIAMEhvg=',
   'x-amzn-trace-id': 'Root=1-65b40104-661be1b329530953236c2b2f'},
  'RetryAttempts': 0},
 'agentActionGroup': {'agentId': 'JBHDO979WZ',
  'agentVersion': 'DRAFT',
  'actionGroupId': 'HRH7ZRR8UR',
  'actionGroupName': 'ClaimManagementActionGroup',
  'description': 'Actions for listing claims, identifying missing paperwork, sending reminders',
  'createdAt': datetime.datetime(2024, 1, 26, 18, 59, 16, 396024, tzinfo=tzutc()),
  'updatedAt': datetime.datetime(2024, 1, 26, 18, 59, 16, 396024, tzinfo=tzutc()),
  'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:626103420214:function:insurance-claims-agent-us-east-1-62610342021

### Allowing Agent to invoke Action Group Lambda
Before using our action group, we need to allow our agent to invoke the lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [20]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

### Preparing Agent
Let's create a DRAFT version of the agent that can be used for internal testing.

In [21]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': '725f27ba-dfc8-4ec9-85af-c5a903723f2f',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 26 Jan 2024 18:59:32 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '725f27ba-dfc8-4ec9-85af-c5a903723f2f',
   'x-amz-apigw-id': 'SKUbTGYsIAMECtA=',
   'x-amzn-trace-id': 'Root=1-65b40114-2f5bc9e91daadb3747c7d52e'},
  'RetryAttempts': 0},
 'agentId': 'JBHDO979WZ',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 1, 26, 18, 59, 32, 891512, tzinfo=tzutc())}

### Create Agent alias
We will now create an alias of the agent that can be used to deploy the agent.

In [22]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)

In [23]:
agent_alias

{'ResponseMetadata': {'RequestId': 'f01295be-b84f-4df8-a900-fb90822b7f98',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 26 Jan 2024 19:00:07 GMT',
   'content-type': 'application/json',
   'content-length': '338',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f01295be-b84f-4df8-a900-fb90822b7f98',
   'x-amz-apigw-id': 'SKUgpEzHoAMEpCA=',
   'x-amzn-trace-id': 'Root=1-65b40137-1d1fc6d736444ae33da50150'},
  'RetryAttempts': 0},
 'agentAlias': {'agentId': 'JBHDO979WZ',
  'agentAliasId': 'FI4IXV1EKM',
  'agentAliasName': 'workshop-alias',
  'agentAliasArn': 'arn:aws:bedrock:us-east-1:626103420214:agent-alias/JBHDO979WZ/FI4IXV1EKM',
  'routingConfiguration': [],
  'createdAt': datetime.datetime(2024, 1, 26, 19, 0, 7, 285418, tzinfo=tzutc()),
  'updatedAt': datetime.datetime(2024, 1, 26, 19, 0, 7, 285418, tzinfo=tzutc()),
  'agentAliasStatus': 'CREATING'}}

### Invoke Agent
Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [24]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False
# Pause to make sure agent alias is ready
time.sleep(30)

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Tell me about claim-857",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-01-26 14:00:46,880] p70823 {992127255.py:21} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 26 Jan 2024 19:00:46 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '249ef072-bc7d-11ee-9560-38f9d3651121',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '1e400a7a-356a-42c6-9b5d-ff22151062d9'},
                      'HTTPStatusCode': 200,
                      'RequestId': '1e400a7a-356a-42c6-9b5d-ff22151062d9',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x115ceaa40>,
 'contentType': 'application/json',
 'sessionId': '249ef072-bc7d-11ee-9560-38f9d3651121'}


In [25]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-01-26 14:00:52,921] p70823 {<timed exec>:11} INFO - {
  "agentId": "JBHDO979WZ",
  "agentAliasId": "FI4IXV1EKM",
  "sessionId": "249ef072-bc7d-11ee-9560-38f9d3651121",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "7feab324-ee7b-4911-9305-0e1215aeefb8-pre-0",
        "text": "You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<tools>\n    <tool_description>\n<tool_name>GET::ClaimManagementActionGroup::getAllOpenClaims</tool_name>\n<description>Gets the list of all open insurance claims. Returns all claimIds that are open.</description>\n<returns>\n<output>\n<type>ar

CPU times: user 23.1 ms, sys: 8.58 ms, total: 31.6 ms
Wall time: 13.3 s


In [26]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

Here are the details for claim claim-857:

Claim ID: claim-857
Created Date: 21-Jul-2023  
Last Activity Date: 25-Jul-2023
Status: Open
Policy Type: Vehicle


### Clean up (optional)
The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:
1. update the action group to disable it
2. delete agent action group
3. delete agent alias
4. delete agent
5. delete lambda function
6. empty created s3 bucket
7. delete s3 bucket

In [27]:
 # This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first

action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [28]:
 agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias['agentAlias']['agentAliasId']
)

In [29]:
 agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [30]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=lambda_name
)

{'ResponseMetadata': {'RequestId': 'b0bcee1f-c697-4362-a573-f6dc881238e5',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Fri, 26 Jan 2024 19:02:14 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b0bcee1f-c697-4362-a573-f6dc881238e5'},
  'RetryAttempts': 0}}

In [31]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'J777KTC15QZ25STC',
  'HostId': 'pOMdoBqBlwV/uIZWuYAl2zYg1jq5wWqkdLk0YDktY0X1RsrVe25xVbWStx96FN+fML8Bli/fSaM=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'pOMdoBqBlwV/uIZWuYAl2zYg1jq5wWqkdLk0YDktY0X1RsrVe25xVbWStx96FN+fML8Bli/fSaM=',
   'x-amz-request-id': 'J777KTC15QZ25STC',
   'date': 'Fri, 26 Jan 2024 19:02:18 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [32]:
# Delete IAM Roles and policies

for policy in [bedrock_agent_bedrock_allow_policy_name, bedrock_agent_s3_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_role_name]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy, agent_s3_schema_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)

## Conclusion
We have now experimented with using `boto3` SDK to create, invoke and delete an agent.

### Take aways
- Adapt this notebook to create new agents for your application

## Thank You